### Binding robot commands - 2022 0107 CJH
---

In [1]:
import wpilib
import commands2

#### you need something to act on (motor, solenoid) or measure
* usually do that in a class that is based on the **Subsystem** class
* a simple one is here with one motor
* put in a function that you can call to do something useful


In [22]:
# get the Subsystem class from the pyfrc library and the Spark motor controller
from commands2 import SubsystemBase
from wpilib import Spark

class Intake(SubsystemBase):
    def __init__(self):
        super().__init__()  # need this so that SubsystemBase is set up correctly (call its __init__)
        
        self.intake_spark = Spark(6) # set up a simple PWM motor
 

    def set_intake(self, speed=0):
        self.intake_spark.set(speed)


#### the simplest way - commands2 lets you lambda them
* can to this directly in oi (pre-2021) or robotcontainer (after 2021) or wherever you want to do the binding

In [27]:
from commands2.button import JoystickButton
import time

class RobotContainer():
    
    def __init__(self):
        
        self.intake = Intake()
        self.start_time = time.time()
        
        self.stick = wpilib.Joystick(port=0)  
        self.buttonA = JoystickButton(self.stick, 1)
        self.buttonB = JoystickButton(self.stick, 2)
        self.buttonX = JoystickButton(self.stick, 3)
        self.buttonY = JoystickButton(self.stick, 4)
        
        # super simple
        self.buttonA.whenPressed(lambda: self.intake.set_intake(speed=0.5))  # send half power
        self.buttonB.whenPressed(lambda: self.intake.set_intake(speed=0))  # stop motor
        
        # I think we can also do this way
        self.buttonX.whenPressed(commands2.InstantCommand(lambda: self.intake.set_intake(speed=0.5), self.intake))
        self.buttonY.whenPressed(commands2.InstantCommand(lambda: self.intake.set_intake(speed=0.0), self.intake))
        

#### the much more verbose way - make a command as a separate class and call it
* make the class file as below, modify as needed
* this gives you total control over every aspect of the command

In [28]:
class SetIntake(commands2.CommandBase):
    def __init__(self, container, speed=0) -> None:
        super().__init__()
        self.setName('set_intake')
        
        self.speed = speed  # take the value we passed in and make it available as a class member variable
        
    def initialize(self) -> None:
        self.start_time = self.container.get_enabled_time()
        print("\n" + f"** Started {self.__class__.__name__} / {self.getName()} at {self.start_time:.1f} s **")
        
        self.container.intake.set_intake(self.speed)  #
        
    def execute(self) -> None:
        pass  # add stuff here if you need to 
    
    def isFinished(self) -> bool:
        return True
    
    def end(self, interrupted: bool) -> None:
        end_time = self.container.get_enabled_time()
        message = 'Interrupted' if interrupted else 'Ended'
        print(f"** {message} {self.getName()} at {end_time:.11f} s after {end_time - self.start_time:.1f} s **")
    

* now RobotContainer has to import the command you wrote 
* replace the lambda with the actual call

In [ ]:
from commands2.button import JoystickButton
import time

class RobotContainer():
    
    def __init__(self):
        
        self.intake = Intake()
        self.start_time = time.time()
        
        self.stick = wpilib.Joystick(port=0)  
        self.buttonA = JoystickButton(self.stick, 1)
        self.buttonB = JoystickButton(self.stick, 2)
        
        self.buttonA.whenPressed(SetIntake(speed=0.5))  # send half power
        self.buttonB.whenPressed(SetIntake(speed=0))  # stop motor
        
    def get_enabled_time(self):  # call when we want to know the start/elapsed time for status and debug messages
        return time.time() - self.start_time